# 导入

In [1]:
import cv2
import albumentations as A
import os
import numpy as np

# 图像填充

In [2]:
def image_padding(image, size=640):
    h, w = image.shape[:2]

    if h >= size or w >= size:
        # Resize the image if any dimension is larger than the target size
        scale = size / max(h, w)
        new_h, new_w = int(h * scale), int(w * scale)
        image = cv2.resize(image, (new_w, new_h))

    h, w = image.shape[:2]
    top = (size - h) // 2
    bottom = size - h - top
    left = (size - w) // 2
    right = size - w - left

    # Ensure non-negative values
    top = max(0, top)
    bottom = max(0, bottom)
    left = max(0, left)
    right = max(0, right)

    padded_image = cv2.copyMakeBorder(image, 
                                      top=top,
                                      bottom=bottom,
                                      left=left,
                                      right=right,
                                      borderType=cv2.BORDER_CONSTANT, 
                                      value=[0, 0, 0])
    return padded_image

# 图像过曝操作

In [3]:
# 将原始图片过曝
def adjust_gamma(image, gamma=1.0):
    """
    Apply gamma correction to the input image.

    Parameters:
    - image: The input image.
    - gamma: The gamma value to adjust the brightness. Values > 1 will make the image brighter.

    Returns:
    - adjusted_image: The gamma-corrected image.
    """
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in np.arange(0, 256)]).astype("uint8")
    adjusted_image = cv2.LUT(image, table)
    return adjusted_image
    

# 运动模糊

In [4]:
def motion_blur(image, size=15, angle=0):
    """
    对图像应用运动模糊效果。
    
    参数:
    - image: 输入图像。
    - size: 模糊核的大小，决定了模糊的强度。
    - angle: 模糊的方向，以度为单位。
    
    返回:
    - 模糊后的图像。
    """
    # 创建运动模糊核
    M = cv2.getRotationMatrix2D((size / 2, size / 2), angle, 1)
    motion_blur_kernel = np.diag(np.ones(size))
    motion_blur_kernel = cv2.warpAffine(motion_blur_kernel, M, (size, size))
    
    motion_blur_kernel = motion_blur_kernel / size
    
    # 应用运动模糊核
    blurred = cv2.filter2D(image, -1, motion_blur_kernel)
    
    # 将核四周的黑边去掉
    cv2.normalize(blurred, blurred, 0, 255, cv2.NORM_MINMAX)
    return blurred

# 数据增强

In [5]:
def augment_image(image, augmenter):
    return augmenter(image=image)['image']

def augment_and_save(image_folder, augmented_folder, num_augments=3):
    if not os.path.exists(augmented_folder):
        os.makedirs(augmented_folder)
    
    # 定义数据增强操作
    augmenter = A.Compose([
        A.HorizontalFlip(p=0.5), # 添加水平翻转
        A.VerticalFlip(p=0.5),  # 添加水平翻转和垂直翻转
        A.RandomRotate90(p=0.5), # 添加90度旋转
        A.ShiftScaleRotate(scale_limit=0.2, rotate_limit=20, shift_limit=0.2, p=0.5), # 添加平移、缩放和旋转变换
        A.RandomBrightnessContrast(p=0.5), # 添加随机亮度和对比度变换
        A.HueSaturationValue(p=0.5), # 添加色调、饱和度、亮度变换
        A.RGBShift(p=0.5), # 添加RGB偏移
        A.CLAHE(p=0.5), # 添加对比度受限自适应直方图均衡化
        A.RandomGamma(p=0.5), # 添加Gamma变换
        A.GaussNoise(p=0.5), # 添加高斯噪声
        A.Blur(p=0.3), # 添加模糊
        A.CoarseDropout(max_holes=8, max_height=64, max_width=64, p=0.5), # 添加遮挡
    ])

    # 定义旋转增强操作
    for image_file in os.listdir(image_folder):
        image_path = os.path.join(image_folder, image_file)
        image = cv2.imread(image_path)
        if image is None:
            continue

        # 对图像进行缩放 640x640
        image = cv2.resize(image, (640, 640))

        # 保存原始图像
        cv2.imwrite(os.path.join(augmented_folder, f"{os.path.splitext(image_file)[0]}_original.jpg"), image)
        
        # 随机触发自定义增强操作
        if np.random.rand() < 0.4:

            # 图像随机缩放比值为
            random_scale = np.random.uniform(0.5, 0.8)
            shrink_image = cv2.resize(image, (int(image.shape[1] * random_scale), int(image.shape[0] * random_scale)))
            shrink_image = image_padding(shrink_image, 640)
            shrink_image_path = os.path.join(augmented_folder, f"{os.path.splitext(image_file)[0]}_shrink.jpg")
            cv2.imwrite(shrink_image_path, shrink_image)

            # 产出过曝图像
            gamma_adjusted_image = adjust_gamma(image, gamma=2.0)
            gamma_adjusted_image_path = os.path.join(augmented_folder, f"{os.path.splitext(image_file)[0]}_gamma.jpg")
            cv2.imwrite(gamma_adjusted_image_path, gamma_adjusted_image)

            # 产出运动模糊图像
            motion_blurred_image = motion_blur(image, size=15, angle=45)
            motion_blurred_image = image_padding(motion_blurred_image, 640)
            motion_blurred_image_path = os.path.join(augmented_folder, f"{os.path.splitext(image_file)[0]}_motion.jpg")
            cv2.imwrite(motion_blurred_image_path, motion_blurred_image)
        
        # 生成其他增强图像
        for i in range(num_augments):
            augmented_image = augment_image(image, augmenter)
            augmented_image = image_padding(augmented_image, 640)
            augmented_image_path = os.path.join(augmented_folder, f"{os.path.splitext(image_file)[0]}_aug_{i}.jpg")
            cv2.imwrite(augmented_image_path, augmented_image)

# Main

In [6]:
# 请求用户输入路径
image_folder = input("请输入图像文件夹的路径: ")
augmented_folder = os.path.join(image_folder, 'augmented')

# 调用函数进行数据增强
augment_and_save(image_folder, augmented_folder, num_augments=3)